##### pip install implicit

use this command in the anaconda prompt before executing this notebook, since this notebook uses the implicit.py package

## BUILDING AND EVALUATING A RECOMMENDER SYSTEM (RS)

This Jupyter Notebook explains how an error metric is generated for weighted matrix factorization. It will go through the data manipulation, splitting of the files, implementation of the WMF RS algorithm and the RS's evaluation process.






In [3]:
#installing packages
import random
import numpy as np
import pandas as pd
import time
import scipy.sparse as sparse
import operator
import implicit

### LOADING THE DATA

In [2]:
#Please enter the path to the input data
#Other input data should look like:
#0,1,3
#0,2,6
#.....
#Where 0 is the user, 1 and 2 are items, and 3 and 6 are implicit feedback scores
#NOTE: Since this is python, the first item and user should both be labeled 0

filepath = 'C:/Users/peter/Documents/uvt/implicit/finalxdata.csv'


In [4]:
def load_matrix(filepath, num_users, num_items):
    counts = np.zeros((num_users, num_items))
    for i, line in enumerate(open(filepath, 'r')):
        user, item, count = line.strip().split(',')
        user = int(user)
        item = int(item)
        count = float(count)
        counts[user][item] = count
    return counts

# So what happens here?
# The formula creates an empty matrix (counts), of size U by I. 
# Then it starts stripping each row of the input file, where it extracts the user, item and feedback score.
# These values are then used to fill the empty matrix with the implicit feedback score.
# Finally it returns the matrix R.


In [5]:
matrix = load_matrix(filepath, num_users = 6518, num_items = 4036)

So now we have loaded the matrix. Let's look at the row of a random user to get a feeling for the data (for this example we'll use user 100). 

In [6]:
item=0
for value in matrix[99]:
    if value>0:
        print(item, value)
    item+=1

690 2.0
1244 2.0
1833 2.0
1882 3.0
1911 2.0
1929 2.0
1941 2.0
2774 2.0


So this user bought 7 different items 2 times, and 1 item 3 times. The code only prints the items on which the user has a purchase record, which means that the user did not purchase the other 4029 items. For good measure, let's look at user 200.

In [7]:
item=0
for value in matrix[199]:
    if value>0:
        print(item, value)
    item+=1

139 2.0
1592 2.0
1806 2.0
1821 2.0
1870 2.0
1931 2.0
2070 2.0


The input data is ready, so lets create a train and test set. First, we'll need to select the users of whom we'll hide one interaction. The following function drops one value in a users row. 

### Creating the test set

In [6]:
# This function tells selects a random user-item-interactions that can be dropped.
# Its input is the user's ID, and the user's implicit scores on all items.
# It returns a list of the user ID, and the ID of the item that will be dropped by another function.
def interactiontodrop(rownumber, row): #rownumber=userid, row=score vector
    scoreditems = []
    items = dict(enumerate(row))
    for k in items:
        if items[k] > 0:
            scoreditems.append(k)
    random.seed(1)
    if not scoreditems:
        print(rownumber)
    return [rownumber, random.choice(scoreditems)]

# This function 'drops' the interactions from the matrix (drop means set equal to zero).
# It lists all users, and then collects a sample of these users.
# For each of these users a random item is selected by the interactiontodrop-function, whereafter it is dropped.
# The input is a matrix, and the output is the list of user-item-interactions that are dropped,
# this list contains the values that are dropped by this function.
def dropper(matrix, n):
    r = list(range(len(matrix)))
    random.seed(1)
    sample = random.sample(r,n)
    toodrop = []
    for x in sample:
        toodrop.append(interactiontodrop(x, matrix[x]))
    for item in toodrop:
        matrix[item[0]][item[1]] = 0
    return toodrop

In [7]:
# So let's check the functions, consider the following matrix X with three users, and four items.
X = [[1,0,0,1],
     [8,0,8,8],
     [9,9,9,9]]

# If we apply the dropper-function to this matrix, it will drop one value in each row, and replace it with zero,
# furthermore, it will remember the user and item id of the dropped interactions

print(dropper(X, 3))
print(X)

[[0, 0], [2, 1], [1, 0]]
[[0, 0, 0, 1], [0, 0, 8, 8], [9, 0, 9, 9]]


So in this example, we have dropped an interaction in each row, namely interaction [0,0], [2,1] and [1,0]
the original matrix X is now 
    X = [[0,0,0,1],
         [0,0,8,8],
         [9,0,9,9]]

With the dropper formula we can create the test set, which usually has the size of 20% of the data,
or in our case 20% of the 6518 users. Therefore, we call the dropper function on the input data with an N of 650.

In [9]:
test_set = dropper(matrix, 650)
# So the test set is now ready and the first 10 user-item interactions in the test set are:
print(test_set[0:10])
# The values of these items and users are now zero in the original matrix
test_user, test_item = test_set[1]
print('the implicit feedback value of the first user in the test set, is now zero')
print(test_user, test_item, matrix[test_user][test_item])

[[1100, 590], [4662, 3351], [6256, 3907], [516, 615], [2089, 1817], [965, 449], [4058, 3394], [6233, 3485], [3682, 3417], [3868, 3376]]
the implicit feedback value of the first user in the test set, is now zero
4662 3351 0.0


With the test set ready, we can now train on the training data and generate recommendations!
The code below fits a WMF model to the data and then calculates the MPR and HLU of this recommendation algorithm. 

### Building and evaluating the RS model

In [10]:
# In order to evaluate our model we need to generate predictions for a user.
# We generate predictions by multiplying the item and user vectors and then sort the items based on.
def prediction(userid, user_vectors, item_vectors, originalmatrix, n=4036):
    # The next line generates the score for a single user.
    predictions = np.dot(user_vectors[userid], item_vectors.T)
    # Since there are also predictions for items the user DID interact with, we need to set these items to zero.
    # (of course these will score highly, since they have positive values in the original matrix)
    for i in range(4036):
        if originalmatrix[userid][i] > 0:
            predictions[i] = -6000
    # Sort all items based on the score, and keep the sorted list of item-ids.         
    dict = {key: value for (key, value) in (enumerate(predictions))}
    sorted_dict = sorted(dict.items(), key=operator.itemgetter(1), reverse=True)
    recommendation = []
    for i in range(n):
        recommendation.append(sorted_dict[i][0])
    return recommendation

def hlu_calc(dropped_items, coefficients, originalmatrix):
    exceptions=0
    model_hlus = []
    user_vectors, item_vectors = coefficients
    for b in dropped_items:
        ranking = (prediction(b[0], user_vectors, item_vectors, originalmatrix).index(b[1]) + 1)
        if ranking < 250:
            ind_hlu = 1 / (2 ** ((ranking) / 10))
            model_hlus.append(ind_hlu)
        else:
            model_hlus.append(0.0000000001)
    return model_hlus

def sparser(originalmatrix, num_users, num_items):
    counts = sparse.dok_matrix((num_users, num_items), dtype=float)
    for i in range(len(originalmatrix)):
        row = originalmatrix[i]
        for j in range(len(row)):
            if row[j] > 0:
                user = int(i)
                item = int(j)
                count = float(row[j])
                counts[user, item] = count
    counts = counts.tocsr()
    return counts

In [11]:
alpha = 10
matrix = matrix*alpha
originalmatrix = matrix
sparsematrix = sparser(matrix.T,  4036, 6518)

lambdaa = 100
model = implicit.als.AlternatingLeastSquares(factors=40, regularization=lambdaa, iterations=20)
model.fit(sparsematrix)
coefficients = [model.user_factors, model.item_factors]
print(np.average(hlu_calc(test_set, coefficients, originalmatrix)))


0.287341019224


The error metric calculation is now done. Below are some small things that I wanted to show. The first box will print the the user and item factors, while the second box shows how we can compute recommendations with these factors. 

In [16]:
print(model.user_factors[1])
print(model.item_factors[1])
# Notice that these values are low, which means that the regularization is doing its job well! :D

[ 0.08465731  0.17181078 -0.00510556  0.04716576  0.04517242 -0.17156409
 -0.00533178  0.15251428  0.02906568 -0.03172248 -0.13372508  0.12144216
  0.1056866  -0.14930712  0.05187108 -0.00961629  0.02596432  0.09711035
 -0.05205087 -0.0819291   0.01326078  0.03573604 -0.05354777 -0.11622727
 -0.02600523 -0.04785348  0.12724786  0.02505602  0.13189727  0.00894016
  0.07240166  0.07903485  0.1500971  -0.03712733 -0.02099241  0.06957538
  0.16389345  0.10668174 -0.04928369 -0.04644141]
[ 0.15259294  0.02953436 -0.28032857  0.28797397 -0.05375853  0.20113707
 -0.07640658  0.03272772  0.19296071  0.0330456  -0.13341269 -0.08293246
 -0.05825168  0.02409248  0.10360389  0.02094175  0.201208   -0.09912423
  0.09220193 -0.15053514 -0.17906176  0.08305001 -0.01685078 -0.03537828
  0.20889714  0.05562918 -0.19676389  0.00037664  0.06571762  0.11021221
  0.03945126 -0.01053052 -0.0167024   0.22189032 -0.08333427 -0.06168398
  0.00448949 -0.04210619  0.04932621 -0.23123421]


So in the example below, were we have user 1, and items 1, 2, and 3, we can see the computed scores for user 1. If we can only recommend 3 items to the user, we would recommend item 2. The user has the highest predicted preference for item 2.

In [15]:
# something goes wrong here, I don't know what. 
# When I initially wrote this notebook, item 2 had the highest score, but later item 3 had the highest score.
# Something probably goes wrong with the seeding here, but it's not that important.
# Just remember, the highest score gets recommended!
print("Item 1 has a score of {}".format(np.dot(model.item_factors[0],model.user_factors[0])))
print("Item 2 has a score of {}".format(np.dot(model.item_factors[1],model.user_factors[0])))
print("Item 3 has a score of {}".format(np.dot(model.item_factors[2],model.user_factors[0]))) 

Item 1 has a score of 0.0038444444071501493
Item 2 has a score of 0.017303235828876495
Item 3 has a score of -0.015008076094090939
